# Working with groups of rows of a data frame

In [ ]:
using DataFrames
using CSV
using Arrow
using Statistics
using FreqTables
using Pipe

using Test

# Adjust table display
ENV["LINES"]   = 15
ENV["COLUMNS"] = 200

# Make sure Chapter 3 is run
@test isfile("auto2.csv")

df = CSV.File("auto2.csv") |> DataFrame

In [ ]:
@test isfile("auto2.arrow")
df2 = Arrow.Table("auto2.arrow") |> DataFrame

In [ ]:
@test df == df2 # no missing in both tables

In general `Arrow.jl` is the preferred way to store data frames, see [Arrow.jl blog](https://bkamins.github.io/julialang/2020/11/06/arrow.html).

Notice that it uses its own `AbstractVector` type:

In [ ]:
@test df2.mpg     isa Arrow.Primitive{Float64, Vector{Float64}}
@test df2[!,:mpg] isa Arrow.Primitive{Float64, Vector{Float64}}
@test !(df2.mpg isa Vector)

# Turn underlying `Arrow.Primitive` into Vector by 
df3 = copy(df2)
@test df3[!,:mpg] isa Vector

Group data frame by `:brand` column:

In [ ]:
#gdf = groupby(df, [:brand,:cylinders])
#gdf[("chevrolet",4)]
gdf = groupby(df, :brand)

In [ ]:
gdf[("ford",)]

In [ ]:
# Calculate aggregates by group
# Pass :mpg to be processed by mean() and save to column "MPG Average"
brand_mpg = combine(gdf, :mpg => mean => "MPG Average" )

In [ ]:
# sort data frame
ENV["LINES"] = 50
sort!(brand_mpg, "MPG Average", rev=true) # high to low

In [ ]:
# Check data consistency
# The :origin of each brand must be the same
# For example, the :origin of all Ford must be 1.0
freqtable(df, :brand, :origin)

In [ ]:
gdf = groupby(df, :brand)
origin_brand = combine(gdf, :origin => (x -> length(unique(x))) => "uniqueness")

# use pipe the above 2 statements can be combined into 1
origin_brand = @pipe df |> groupby(_, :brand) |> combine(_, :origin => (x -> length(unique(x))) => "uniqueness")


In [ ]:
# Expect min == max == 1
@test (1,1) == extrema(origin_brand[!,"uniqueness"])

In [ ]:
# Group by 2 columns, each brand should have 1 group because there should be 1 origin
origin_brand2 = @pipe df |> groupby(_, [:origin, :brand]) |> combine(_, nrow)

In [ ]:
# unstack to get similar table from freqtable
origin_vs_brand = unstack(origin_brand2, :brand, :origin, :nrow)

In [ ]:
# missing -> 0
# coalesce + broadcast goes through each elemenmt of the data frame
coalesce.(origin_vs_brand, 0)

In [ ]:
names(origin_vs_brand)

In [ ]:
propertynames(origin_vs_brand)

In [ ]:
origin_vs_brand."1.0"
#origin_vs_brand[!,"1.0"]

In [ ]:
# Columns of data frame can hold any data type
#
# Note the use of Ref() is to wrap the resulting array into 1 object.  This prevents the following broadcasting
# of the elements of the array.
origin_brand3 = @pipe df |> groupby(_, :origin) |> combine(_, :brand => (x -> Ref(unique(x))) => "brands") 

In [ ]:
#origin_brand3[1, "brands"]

In [ ]:
# This is not what we wanted
# Without Ref() the result from unique() is broadcasted
# Comment out to find out
#@pipe df |> groupby(_, :origin) |> combine(_, :brand => (x -> unique(x)) => "brands") 

In [ ]:
#origin_brand3."brands"[[1,2]]

In [ ]:
d = origin_brand3
for i in axes(d, 1) # 1:nrow(origin_brand3)
    for j in i+1:nrow(d)
        # 3 ways
        Ans = intersect(d[i,"brands"], d[j,"brands"])
        #Ans = intersect(d[[i,j],"brands"]...)
        #Ans = intersect(d."brands"[[i,j]]...)
                
        println("$i vs $j => $Ans")
    end
end